# Workshop part 3 | Learn how to perform a backtest
In the third part of this workshop, we will perform a backtest for the same location as the first two parts.

The learning points are:
- What a backtest is and how it works on a high level;
- Hands on experience with evaluating a model using a backtest;
- Being able to understand the results of a backtest.

A backtest is the evaluation of the model on historical data. Essentially, it is a way of testing how OpenSTEF would have performed if it had been used in the past. 

In [6]:
import pandas as pd 
import openstef
from openstef.data_classes.model_specifications import ModelSpecificationDataClass
from openstef.data_classes.prediction_job import PredictionJobDataClass 
from openstef.pipeline.train_create_forecast_backtest import train_model_and_forecast_back_test

# Set plotly as the default pandas plotting backend
pd.options.plotting.backend = 'plotly'

## Define the prediction job
The same as in workshop parts 1 and 2, a prediction job has to be defined. As we are making a backttest for same location, we can use the exact same prediction job. 

You can find the documentation [here](https://github.com/OpenSTEF/openstef/blob/main/openstef/data_classes/prediction_job.py).

In [7]:
# Define properties of training/prediction. We call this a 'prediction_job'. The same is used as in the first exercise.
pj = dict(id=287,
        model='xgb', 
        quantiles=[0.10,0.30,0.50,0.70,0.90],
        forecast_type="demand", 
        lat=52.0,
        lon=5.0,
        horizon_minutes=0.25,
        resolution_minutes=15,
        name="workshop_exercise_2",
        save_train_forecasts=True,
       )

pj=PredictionJobDataClass(**pj)
modelspecs = ModelSpecificationDataClass(id=pj['id'])

In [8]:
input_data=pd.read_csv("../data/input_data_sun_heavy.csv", index_col=0, parse_dates=True)

## Perform the backtest
The prediction job and input data have been provided above, so now a backtest can be performed. 

Exercise: 
- Train a model and make a backtest uing one OpenSTEF pipeline 

    - Hint 1: find the correct pipeline on the OpenSTEF [website](https://openstef.github.io/openstef/user_guides.html);

    - Hint 2: You only need 1 pipeline to train the model and make a forecast!


Note: The training_horizons is the horizon of the desired forecast in minutes. It entails how far into the future we want to predict. The value of 15 entails that at the moment of prediction, you predict 15 minutes into the future. So let's say you make a prediction at one o'clock, than the prediction is for 13.15 o'clock.

In [9]:
n_folds=1

forecast, model, train_data, validation_data, test_data = openstef.pipeline. ... (
    ...,
    modelspecs = modelspecs,
    input_data = ...,
    training_horizons=[0.25, 47.0],
    n_folds=n_folds,
 )

2024-02-06 15:31:51 [info     ] Found 194 values of constant load (repeated values), converted to NaN value. cleansing_step=repeated_values frac_values=0.01683881607499349 num_values=194 pj_id=287
2024-02-06 15:31:51 [info     ] Removed 194 NaN values         num_removed_values=194
2024-02-06 15:31:54 [info     ] Postproces in preparation of storing


## Evaluate the results 
Below, the results from the backtest are plotted. With these plots, answer the question on the exercise below. 

Exercise: answer the following questions: 
- When is the model uncertain? Why? 
- What differences do you see between the horizons? 
- Look at the differences between the forecast and realized? What do you see? Why? 

Bonus: look at the differences between the two time horizons using metrics. You can use the build-in metrics package of OpenSTEF. Find a suitable metric and implement this in the code. See the documentation website [here](https://openstef.github.io/openstef/openstef.metrics.html).Does the difference in metrics confirm your answers given above?


In [10]:
for horizon in set(forecast.horizon):
    fig = forecast.loc[forecast.horizon==0.25,['quantile_P10','quantile_P30',
                    'quantile_P50','quantile_P70','quantile_P90','realised','forecast']].plot(
                                                                                   title=f"Horizon: {horizon}")
    fig.update_traces(
         line=dict(color="green", width=1), fill='tonexty', fillcolor='rgba(0, 255, 0, 0.1)',
         selector=lambda x: 'quantile' in x.name and x.name != 'quantile_P10')
    fig.update_traces(
         line=dict(color="green", width=1),
         selector=lambda x: 'quantile_P10' == x.name)
    fig.update_traces(
         line=dict(color="red", width=2),
         selector=lambda x: 'realised' in x.name)
    fig.update_traces(
         line=dict(color="blue", width=2),
         selector=lambda x: 'forecast' in x.name)
    fig.show()

In [ ]:
for horizon in set(forecast.horizon):
     score=openstef. ...  (forecast.loc[forecast.horizon==horizon, 'realised'], forecast.loc[forecast.horizon==horizon, 'forecast'])
     print(horizon, score)